In [ ]:
import numpy as np
# to display full text
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [ ]:
df = pd.read_csv('graph-datav4.csv')

In [ ]:
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
df['user_id'].nunique()

In [ ]:
df[df['user_id'] <= 2]

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
print(sum(df.user_id.isna()))
print(sum(df.text.isna()))
print(sum(df.favorites_count.isna()))
print(sum(df.verified.isna()))
print(sum(df.followers_count.isna()))

In [ ]:
df.text.fillna('nothing to say', inplace = True)

In [ ]:
print(sum(df.user_id.isna()))
print(sum(df.text.isna()))
print(sum(df.favorites_count.isna()))
print(sum(df.verified.isna()))
print(sum(df.followers_count.isna()))

In [ ]:
df['user_id'].nunique()

## Word Embeddings using gensim

In [ ]:
import gensim

In [ ]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
text = list()
line = df['text'].values.tolist()
for l in line:
    print(l)
    token = word_tokenize(l)
    token = [t.lower() for t in token]
    punct = str.maketrans('','',string.punctuation)
    translate = [t.translate(punct) for t in token]
    words = [word for word in translate if word.isalpha()]
    stop = set(stopwords.words('english'))
    words = [t for t in words if not t in stop]
    text.append(words)

In [ ]:
len(text)

In [ ]:
model = gensim.models.Word2Vec(sentences = text, size = 100, window = 2, workers = 3, min_count = 1)

In [ ]:
words = list(model.wv.vocab)
print('Vocab size', len(words))

In [ ]:
words

## Most Similar Words

In [ ]:
model.wv.most_similar('tweet')

## Save vectors

In [ ]:
max_length = max([len(s.split()) for s in df.text])

In [ ]:
max_length

In [ ]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [ ]:
file = 'twitter_charlie_manual_word2vec_graph.txt'

In [ ]:
model.wv.save_word2vec_format(file,binary = False)

In [ ]:
import os
emb = {}
f = open(os.path.join('', 'twitter_charlie_manual_word2vec_graph.txt'), encoding = 'utf-8')
for line in f:
    values = line.split()
    word = values[0]
    value = np.asarray(values[1:])
    emb[word] = value
f.close()

In [ ]:
token = Tokenizer()
token.fit_on_texts(text)
seq = token.texts_to_sequences(text)
index = token.word_index
print('unique no of tokens ', index)
padding = pad_sequences(seq, maxlen = max_length)
rumour = df['label'].values

In [ ]:
padding[13]

In [ ]:
n = len(index) + 1
embed_mat = np.zeros((n, 100))
for s,values in index.items():
    print(s,values)
    if values>n:
        continue
    embed_vec = emb.get(s)
    if embed_vec is not None:
        embed_mat[values] = embed_vec

In [ ]:
embed_mat.shape

In [ ]:
len(index)

In [ ]:
len(padding)

In [ ]:
sorted_words = sorted(index)
#print(sorted_words)

In [ ]:
index.keys()

## Map words with their vectors

In [ ]:
features = {}

In [ ]:
d = np.arange(0, len(padding))

In [ ]:
data = np.zeros((0, len(df)))

In [ ]:
feat_df = pd.DataFrame(index = d, columns = sorted_words)
#print(feat_df)

In [ ]:
feat_df.shape

In [ ]:
feat_df.fillna(0.0, inplace = True)

In [ ]:
feat_df.head()

In [ ]:
#combining features with average
for row in range(0, len(padding)):
    print(row)
    for number in padding[row]:
        for key,value in index.items():
            if number == value:
                mean = embed_mat[number].mean()
                if (feat_df.iloc[row][key] == 0.0):
                    feat_df.loc[row,key] = mean
                
                else:
                    feat_df.loc[row,key] = feat_df.iloc[row][key] + mean
                break

In [ ]:
feat_df.to_csv('features of word2vec embeddings.csv')

In [ ]:
feat_df = pd.read_csv('features of word2vec embeddings.csv')
feat_df.head()

In [ ]:
feat_df.info()

In [ ]:
feat_df=feat_df.fillna(0)

In [ ]:
feat_df.shape

## Combining Features into 1 dataframe

In [ ]:
df.drop('text', axis = 1, inplace = True)

In [ ]:
df.head()

In [ ]:
X_new = pd.concat([feat_df, df], axis=1)

In [ ]:
X_new.shape

In [ ]:
X_new.fillna(0)

In [ ]:
X_new.to_csv('charlie_combined_word2vec_features_with_original_features.csv')

In [ ]:
import pandas as pd
checking = pd.read_csv('charlie_combined_word2vec_features_with_original_features.csv')

In [ ]:
checking['user_id'].nunique()

In [ ]:
checking.shape

In [ ]:
checking[checking['user_id'] == 0]